In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, current_date, to_date, lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

25/08/14 07:46:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
# spark.conf.set("spark.sql.adaptive.enabled", "false")
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [4]:
billing_events = spark.read.format("csv").option("header", "true").csv("/home/iceberg/data/billing_events.csv").withColumn("event_date", expr("CAST(DATE_TRUNC('day', event_date) AS DATE)")).select("subscription_id", "event_id", "event_type", "amount", "event_date")
subscriptions = spark.read.format("csv").option("header", "true").csv("/home/iceberg/data/subscriptions.csv").select("subscription_id", "customer_id", "end_date", "plan_id")
customers = spark.read.format("csv").option("header", "true").csv("/home/iceberg/data/customers.csv").select("customer_id", "name", "status", "signup_date", "country")

In [5]:
billing_events.rdd.getNumPartitions()

3

In [6]:
subscriptions.rdd.getNumPartitions()

1

In [7]:
customers.rdd.getNumPartitions()

1

In [8]:
billing_events.explain()

== Physical Plan ==
*(1) Project [subscription_id#18, event_id#17, event_type#19, amount#20, cast(date_trunc(day, cast(event_date#22 as timestamp), Some(Etc/UTC)) as date) AS event_date#31]
+- FileScan csv [event_id#17,subscription_id#18,event_type#19,amount#20,event_date#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/billing_events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<event_id:string,subscription_id:string,event_type:string,amount:string,event_date:string>




In [9]:
billing_events.show()
subscriptions.show()
customers.show()

+--------------------+--------------------+-----------------+------+----------+
|     subscription_id|            event_id|       event_type|amount|event_date|
+--------------------+--------------------+-----------------+------+----------+
|24773376-c839-499...|3305b836-cfa8-46c...|  invoice_created|  50.0|2023-12-26|
|24773376-c839-499...|16ce87b0-2a48-468...|payment_succeeded|  50.0|2023-12-27|
|24773376-c839-499...|034a8ee9-745f-4fd...|  invoice_created|  50.0|2024-01-25|
|24773376-c839-499...|f90f989d-05a2-49f...|payment_succeeded|  50.0|2024-01-28|
|24773376-c839-499...|1c46002f-579f-415...|  invoice_created|  20.0|2024-02-24|
|24773376-c839-499...|56bfcb32-53f0-46b...|   payment_failed|  20.0|2024-02-25|
|e95343db-47ca-426...|a2409c65-1384-472...|  invoice_created|  20.0|2021-06-04|
|e95343db-47ca-426...|bfaa2f89-504d-46e...|payment_succeeded|  20.0|2021-06-06|
|e95343db-47ca-426...|b0891ca8-0a49-4ff...|  invoice_created|  20.0|2021-07-05|
|e95343db-47ca-426...|4b296f09-ea22-4f2.

In [10]:
df = billing_events.join(subscriptions, on="subscription_id", how="inner").join(customers, on="customer_id", how="inner")

In [11]:
df.rdd.getNumPartitions()

3

In [12]:
df.show()

+--------------------+--------------------+--------------------+-----------------+------+----------+----------+--------------------+----------------+------+-----------+-------------+
|         customer_id|     subscription_id|            event_id|       event_type|amount|event_date|  end_date|             plan_id|            name|status|signup_date|      country|
+--------------------+--------------------+--------------------+-----------------+------+----------+----------+--------------------+----------------+------+-----------+-------------+
|ed43e3e4-c1f1-4d7...|24773376-c839-499...|3305b836-cfa8-46c...|  invoice_created|  50.0|2023-12-26|2024-03-21|c55ebfae-af13-45e...|      Brian Yang|active| 2023-08-02|United States|
|ed43e3e4-c1f1-4d7...|24773376-c839-499...|16ce87b0-2a48-468...|payment_succeeded|  50.0|2023-12-27|2024-03-21|c55ebfae-af13-45e...|      Brian Yang|active| 2023-08-02|United States|
|ed43e3e4-c1f1-4d7...|24773376-c839-499...|034a8ee9-745f-4fd...|  invoice_created|  5

In [13]:
billing_events.join(subscriptions, on="subscription_id", how="inner").join(customers, on="customer_id", how="inner").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#63, subscription_id#18, event_id#17, event_type#19, amount#20, event_date#31, end_date#66, plan_id#64, name#101, status#104, signup_date#102, country#103]
   +- BroadcastHashJoin [customer_id#63], [customer_id#98], Inner, BuildRight, false
      :- Project [subscription_id#18, event_id#17, event_type#19, amount#20, event_date#31, customer_id#63, end_date#66, plan_id#64]
      :  +- BroadcastHashJoin [subscription_id#18], [subscription_id#62], Inner, BuildRight, false
      :     :- Project [subscription_id#18, event_id#17, event_type#19, amount#20, cast(date_trunc(day, cast(event_date#22 as timestamp), Some(Etc/UTC)) as date) AS event_date#31]
      :     :  +- Filter isnotnull(subscription_id#18)
      :     :     +- FileScan csv [event_id#17,subscription_id#18,event_type#19,amount#20,event_date#22] Batched: false, DataFilters: [isnotnull(subscription_id#18)], Format: CSV, Location: InMemoryFileIndex(1 pat

In [14]:
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760)  # default is 10MB

In [15]:
%%sql

CREATE DATABASE IF NOT EXISTS saas

++
||
++
++

In [16]:
%%sql

CREATE TABLE IF NOT EXISTS saas.fct_billing_events (
    customer_id STRING,
    dim_name STRING,
    dim_customer_status STRING,
    dim_signup_date DATE,
    dim_subscription_end_date DATE,
    subscription_id STRING,
    event_id STRING,
    dim_event_type STRING,
    m_amount DOUBLE,
    plan_id STRING,
    dim_event_date DATE,
    dim_country STRING,
    month_start DATE,
    ds DATE
) 
USING iceberg
PARTITIONED BY (months(dim_event_date));

++
||
++
++

In [17]:
%%sql

CREATE TABLE IF NOT EXISTS saas.fct_billing_events_unsorted (
    customer_id STRING,
    dim_name STRING,
    dim_customer_status STRING,
    dim_signup_date DATE,
    dim_subscription_end_date DATE,
    subscription_id STRING,
    event_id STRING,
    dim_event_type STRING,
    m_amount DOUBLE,
    plan_id STRING,
    dim_event_date DATE,
    dim_country STRING,
    month_start DATE,
    ds DATE
) 
USING iceberg
PARTITIONED BY (months(dim_event_date));

++
||
++
++

In [18]:
df_new = df.select(
    col("customer_id"),
    col("name").alias("dim_name"),
    col("status").alias("dim_customer_status"),
    col("signup_date").cast("date").alias("dim_signup_date"),
    col("end_date").cast("date").alias("dim_subscription_end_date"),
    col("subscription_id"),
    col("event_id"),
    col("event_type").alias("dim_event_type"),
    col("amount").cast("double").alias("m_amount"),
    col("plan_id"),
    to_date(col("event_date"), "yyyy-MM-dd").alias("dim_event_date"),
    col("country").alias("dim_country"),
    expr("CAST(DATE_TRUNC('month', event_date) AS DATE)").alias("month_start"),
    current_date().alias("ds")
)

In [19]:
# spark.conf.set("spark.sql.adaptive.enabled", "true")

In [20]:
df_new = df_new.repartition(expr("date_trunc('month', dim_event_date)"))

In [21]:
df_new.rdd.getNumPartitions()

[Stage 13:===================>                                      (1 + 2) / 3]

7

In [22]:
df_sorted = df_new.sortWithinPartitions(col("dim_event_type"), col("dim_customer_status"), col("month_start"))

In [23]:
# df_sorted = df_new.sortWithinPartitions(col("dim_event_type"), col("dim_customer_status"), )
# df_new.write.mode("overwrite").saveAsTable("saas.fct_billing_events_unsorted")
# df_sorted.write.mode("overwrite").saveAsTable("saas.fct_billing_events")

In [24]:
# Option B: DataFrame API with Iceberg-specific writeTo
(df_sorted
   .writeTo("saas.fct_billing_events")
   .overwritePartitions()       # for full overwrite
   # .append()                  # if you just want to add new data
)

In [25]:
# Option B: DataFrame API with Iceberg-specific writeTo
(df_new
   .writeTo("saas.fct_billing_events_unsorted")
   .overwritePartitions()       # for full overwrite
   # .append()                  # if you just want to add new data
)

In [26]:
df_sorted.select(col("dim_customer_status"), col("dim_event_type"), col("dim_event_date")).show(5)

+-------------------+---------------+--------------+
|dim_customer_status| dim_event_type|dim_event_date|
+-------------------+---------------+--------------+
|             active|invoice_created|    2020-01-21|
|             active|invoice_created|    2020-01-22|
|             active|invoice_created|    2020-01-30|
|             active|invoice_created|    2020-01-29|
|             active|invoice_created|    2020-01-30|
+-------------------+---------------+--------------+
only showing top 5 rows



In [27]:
%%sql

SELECT SUM(file_size_in_bytes) AS size, 'sorted'
FROM saas.fct_billing_events.files

UNION ALL

SELECT SUM(file_size_in_bytes) AS size, 'unsorted'
FROM saas.fct_billing_events_unsorted.files

size,sorted
3696931,sorted
3727141,unsorted


In [29]:
%%sql

SELECT * FROM saas.fct_billing_events.files LIMIT 3

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/saas/fct_billing_events/data/dim_event_date_month=2022-10/00000-28-d51a1584-e62b-490d-92e4-17defb6e3437-0-00003.parquet,PARQUET,0,Row(dim_event_date_month=633),1327,75690,"{1: 14378, 2: 6401, 3: 134, 4: 2655, 5: 2421, 6: 14697, 7: 26950, 8: 155, 9: 746, 10: 646, 11: 978, 12: 1060, 13: 80, 14: 73}","{1: 1327, 2: 1327, 3: 1327, 4: 1327, 5: 1327, 6: 1327, 7: 1327, 8: 1327, 9: 1327, 10: 1327, 11: 1327, 12: 1327, 13: 1327, 14: 1327}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 157, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}",{9: 0},"{1: bytearray(b'0005385d-26ae-46'), 2: bytearray(b'Abigail Case'), 3: bytearray(b'active'), 4: bytearray(b'WG\x00\x00'), 5: bytearray(b'SK\x00\x00'), 6: bytearray(b'000613e1-4efd-42'), 7: bytearray(b'00181c60-d230-4e'), 8: bytearray(b'invoice_created'), 9: bytearray(b'\x85\xebQ\xb8\x1eUW\xc0'), 10: bytearray(b'2e320402-1f28-49'), 11: bytearray(b'BK\x00\x00'), 12: bytearray(b'Australia'), 13: bytearray(b'BK\x00\x00'), 14: bytearray(b'UO\x00\x00')}","{1: bytearray(b'ffb597d8-a10d-46'), 2: bytearray(b'Yolanda Mccoy'), 3: bytearray(b'inactive'), 4: bytearray(b'PK\x00\x00'), 5: bytearray(b'{O\x00\x00'), 6: bytearray(b'ff9abfdb-cc0c-44'), 7: bytearray(b'fffc00bc-f8f0-45'), 8: bytearray(b'refund'), 9: bytearray(b'\xf6(\\\x8f\xc2\x8d\x90@'), 10: bytearray(b'eb19d54f-75b5-4f'), 11: bytearray(b'`K\x00\x00'), 12: bytearray(b'United States'), 13: bytearray(b'BK\x00\x00'), 14: bytearray(b'UO\x00\x00')}",None,[4],None,0,None,None,None,"Row(customer_id=Row(column_size=14378, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='0005385d-26ae-46', upper_bound='ffb597d8-a10d-46'), dim_country=Row(column_size=1060, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='Australia', upper_bound='United States'), dim_customer_status=Row(column_size=134, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='active', upper_bound='inactive'), dim_event_date=Row(column_size=978, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound=datetime.date(2022, 10, 1), upper_bound=datetime.date(2022, 10, 31)), dim_event_type=Row(column_size=155, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='invoice_created', upper_bound='refund'), dim_name=Row(column_size=6401, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='Abigail Case', upper_bound='Yolanda Mccoy'), dim_signup_date=Row(column_size=2655, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound=datetime.date(2020, 1, 2), upper_bound=datetime.date(2022, 10, 15)), dim_subscription_end_date=Row(column_size=2421, value_count=1327, null_value_count=157, nan_value_count=None, lower_bound=datetime.date(2022, 10, 18), upper_bound=datetime.date(2025, 9, 16)), ds=Row(column_size=73, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound=datetime.date(2025, 8, 9), upper_bound=datetime.date(2025, 8, 9)), event_id=Row(column_size=26950, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='00181c60-d230-4e', upper_bound='fffc00bc-f8f0-45'), m_amount=Row(column_size=746, value_count=1327, null_value_count=0, nan_value_count=0, lower_bound=-93.33, upper_bound=1059.44), month_start=Row(column_size=80, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound=datetime.date(2022, 10, 1), upper_bound=datetime.date(2022, 10, 1)), plan_id=Row(column_size=646, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='2e320402-1f28-49', upper_bound='eb19d54f-75b5-4f'), subscription_id=Row(column_size=14697, value_count=1327, null_value_count=0, nan_value_count=None, lower_bound='000613e1-4efd-42', u

In [30]:
billing_events

DataFrame[subscription_id: string, event_id: string, event_type: string, amount: string, event_date: date]

In [94]:
%%sql

CREATE TABLE IF NOT EXISTS saas.billing_events_bucketed (
    subscription_id STRING,
    event_id STRING,
    event_type STRING,
    amount STRING,
    event_date DATE
)
USING iceberg
PARTITIONED BY (bucket(16, subscription_id));

++
||
++
++

In [95]:
subscriptions

DataFrame[subscription_id: string, customer_id: string, end_date: string, plan_id: string]

In [96]:
%%sql

CREATE TABLE IF NOT EXISTS saas.subscriptions_bucketed (
    subscription_id STRING,
    customer_id STRING,
    end_date STRING,
    plan_id STRING
)
USING iceberg
PARTITIONED BY (bucket(16, subscription_id));

++
||
++
++

In [97]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [98]:
# billing_events.writeTo("saas.billing_events_bucketed").overwrite(lit(True))

In [99]:
billing_events.write.mode("overwrite").bucketBy(16, "subscription_id").saveAsTable("saas.billing_events_bucketed")

In [100]:
%%sql
    
SELECT
  file_path,
  record_count,
  partition.subscription_id_bucket AS bucket_id
FROM saas.billing_events_bucketed.files
ORDER BY bucket_id;

file_path,record_count,bucket_id
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=0/00002-108-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00006.parquet,3943,0
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=1/00000-106-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00002.parquet,3438,1
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=2/00002-108-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00005.parquet,4295,2
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=3/00000-106-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00005.parquet,4020,3
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=4/00001-107-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00004.parquet,4489,4
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=5/00001-107-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00001.parquet,3941,5
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=6/00000-106-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00004.parquet,4246,6
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=7/00002-108-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00001.parquet,3849,7
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=8/00001-107-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00005.parquet,4431,8
s3://warehouse/saas/billing_events_bucketed/data/subscription_id_bucket=9/00001-107-c0afbd89-7ebf-488b-8bb8-c7398bdef5f6-0-00003.parquet,3994,9


In [101]:
subscriptions.write.mode("overwrite").bucketBy(16, "subscription_id").saveAsTable("saas.subscriptions_bucketed")

In [102]:
%%sql

SELECT 
    file_path,
    record_count,
    partition.subscription_id_bucket AS bucket_id
FROM saas.subscriptions_bucketed.files
ORDER BY bucket_id;

file_path,record_count,bucket_id
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=0/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00016.parquet,395,0
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=1/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00002.parquet,366,1
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=2/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00015.parquet,352,2
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=3/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00005.parquet,393,3
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=4/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00009.parquet,389,4
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=5/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00006.parquet,373,5
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=6/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00004.parquet,356,6
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=7/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00011.parquet,382,7
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=8/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00010.parquet,380,8
s3://warehouse/saas/subscriptions_bucketed/data/subscription_id_bucket=9/00000-111-af4fa6fc-acf7-4924-9e8c-b4e73df7ed02-0-00008.parquet,377,9


In [103]:
billing_events.createOrReplaceTempView("be")
subscriptions.createOrReplaceTempView("s")

In [ ]:
spark.sql("""
    SELECT * FROM saas.billing_events_bucketed beb JOIN saas.subscriptions_bucketed sb
    ON beb.subscription_id = sb.subscription_id    
""").explain()

In [105]:
spark.sql("""
    SELECT * FROM be JOIN s
    ON be.subscription_id = s.subscription_id       
""").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [subscription_id#18], [subscription_id#62], Inner
   :- Sort [subscription_id#18 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(subscription_id#18, 200), ENSURE_REQUIREMENTS, [plan_id=2252]
   :     +- Project [subscription_id#18, event_id#17, event_type#19, amount#20, cast(date_trunc(day, cast(event_date#22 as timestamp), Some(Etc/UTC)) as date) AS event_date#31]
   :        +- Filter isnotnull(subscription_id#18)
   :           +- FileScan csv [event_id#17,subscription_id#18,event_type#19,amount#20,event_date#22] Batched: false, DataFilters: [isnotnull(subscription_id#18)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/billing_events.csv], PartitionFilters: [], PushedFilters: [IsNotNull(subscription_id)], ReadSchema: struct<event_id:string,subscription_id:string,event_type:string,amount:string,event_date:string>
   +- Sort [subscription_id#62 ASC NULLS FIRST], false, 0
